In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from pandas.plotting import scatter_matrix
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from scipy.optimize import minimize
from statsmodels.tsa.seasonal import seasonal_decompose
import re
import plotly.express as px
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

"""for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))"""

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [30]:
## !pip install 'polars[numpy,pandas,pyarrow]' --target=/kaggle/working/mysitepackages

!pip download polars[numpy,pandas,pyarrow] -d /kaggle/working/mysitepackages/polars_pkg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 50.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 43.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 99.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 82.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 38.1 MB/s eta 0:00:00
Saved ./mysitepackages/polars_pkg/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
Saved ./mysitepackages/polars_pkg/pyarrow-11.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./mysitepackages/polars_pkg/typing_extensions-4.5.0-py3-none-any.whl
Saved ./mysitepackages/polars_pkg/pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./mysitepackages/polars_pkg/polars-0.17.10-cp37-abi3-manylinux_2_17_x86_64.m

In [31]:
!pip download duckdb==0.7.1 -d /kaggle/working/mysitepackages/duck

  Using cached duckdb-0.7.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (15.2 MB)
Saved ./mysitepackages/duck/duckdb-0.7.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Successfully downloaded duckdb


#### create the zip file with the code below and then download the zip file into local machine

In [32]:
from zipfile import ZipFile

dirName = "/kaggle/working/mysitepackages"
zipName = "packages.zip"

# Create a ZipFile Object
with ZipFile(zipName, 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(dirName):
        for filename in filenames:
            if (filename != zipName):
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # Add file to zip
                zipObj.write(filePath)

In [3]:
! ls /kaggle/input/packages-to-install-offline/kaggle/working/mysitepackages

duck
duckdb-0.7.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
polars_pkg


#### Download the zipfile into local machine

#### Upload the downloaded zipfile from local machine and then fire the command below

In [36]:
!pip install polars[numpy,pandas,pyarrow] --no-index --find-links=file:///kaggle/input/packages-to-install-offline/kaggle/working/mysitepackages/polars_pkg

Looking in links: file:///kaggle/input/packages-to-install-offline/kaggle/working/mysitepackages/polars_pkg


In [4]:
!pip install duckdb --no-index --find-links=file:///kaggle/input/packages-to-install-offline/kaggle/working/mysitepackages/duck

Looking in links: file:///kaggle/input/packages-to-install-offline/kaggle/working/mysitepackages/duck
Processing /kaggle/input/packages-to-install-offline/kaggle/working/mysitepackages/duck/duckdb-0.7.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [5]:
import polars as pl
import duckdb as dd

In [9]:
print(defog_train_data.select('Id').n_unique())
print(defog_train_data.estimated_size("mb"))
print(defog_train_data.shape)

91
291.12402725219727
(4111322, 10)


In [10]:
defog_train_data.head(10)

Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task,Id
i64,f64,f64,f64,i64,i64,i64,bool,bool,str
1000,-1.015138,-0.071969,0.161124,0,0,0,true,true,"""be9d33541d"""
1001,-1.018566,-0.05785,0.144988,0,0,0,true,true,"""be9d33541d"""
1002,-1.037102,-0.0542,0.129327,0,0,0,true,true,"""be9d33541d"""
1003,-1.036416,-0.060578,0.115451,0,0,0,true,true,"""be9d33541d"""
1004,-1.012407,-0.069198,0.110121,0,0,0,true,true,"""be9d33541d"""
1005,-0.98234,-0.078566,0.113947,0,0,0,true,true,"""be9d33541d"""
1006,-0.962148,-0.088102,0.121993,0,0,0,true,true,"""be9d33541d"""
1007,-0.947028,-0.100937,0.129277,0,0,0,true,true,"""be9d33541d"""
1008,-0.938086,-0.105582,0.136236,0,0,0,true,true,"""be9d33541d"""


In [6]:
defog_metadata = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv", try_parse_dates=True)\
.collect()
print(defog_metadata.shape)
print(defog_metadata.select('Subject').n_unique())
print(defog_metadata.select('Id').n_unique())
defog_metadata.filter(pl.col('Subject')=='bf608b')

(137, 4)
45
137


Id,Subject,Visit,Medication
str,str,i64,str
"""02ea782681""","""bf608b""",2,"""on"""
"""4c0aec9035""","""bf608b""",2,"""off"""
"""8228c6fdee""","""bf608b""",1,"""on"""
"""a8c5a489bd""","""bf608b""",1,"""off"""


In [7]:
subjects = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/subjects.csv", try_parse_dates=True)\
.collect()
print(subjects.shape)
print(subjects.select('Subject').n_unique())
subjects.filter(pl.col('Subject')=='bf608b')

(173, 8)
136


Subject,Visit,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
str,i64,i64,str,f64,i64,i64,i64
"""bf608b""",1,67,"""M""",7.0,15,48,23
"""bf608b""",2,67,"""M""",7.0,14,57,19


In [8]:
defog_subjects_metadata = defog_metadata.join(subjects, on=["Subject","Visit"], how="inner")

defog_subjects_metadata = defog_subjects_metadata.with_columns(pl.when(pl.col('Medication') == 'on').then(pl.col('UPDRSIII_On')).otherwise(pl.col('UPDRSIII_Off')).alias("UPDRSIII"))
defog_subjects_metadata = defog_subjects_metadata.drop(['UPDRSIII_On','UPDRSIII_Off'])

del defog_metadata, subjects

print(defog_subjects_metadata.shape)
print(defog_subjects_metadata.select('Subject').n_unique())
print(defog_subjects_metadata.select('Id').n_unique())
defog_subjects_metadata.filter(pl.col('Subject')=='bf608b')

(137, 9)
45
137


Id,Subject,Visit,Medication,Age,Sex,YearsSinceDx,NFOGQ,UPDRSIII
str,str,i64,str,i64,str,f64,i64,i64
"""8228c6fdee""","""bf608b""",1,"""on""",67,"""M""",7.0,23,15
"""a8c5a489bd""","""bf608b""",1,"""off""",67,"""M""",7.0,23,48
"""02ea782681""","""bf608b""",2,"""on""",67,"""M""",7.0,19,14
"""4c0aec9035""","""bf608b""",2,"""off""",67,"""M""",7.0,19,57


In [9]:
events = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/events.csv", try_parse_dates=True)
#.collect()
events = events.with_columns(
    [
        ((pl.col('Init')*100).round(0).cast(pl.Int64)).alias('Init_m'), 
        ((pl.col('Completion')*100).round(0).cast(pl.Int64)).alias('Completion_m')
    ]
)

events = events.with_columns(
    [
        (pl.when(pl.col('Type')=='Turn').then(pl.lit(1)).otherwise(pl.lit(0)).alias('Turn')),
        (pl.when(pl.col('Type')=='Walking').then(pl.lit(1)).otherwise(pl.lit(0)).alias('Walking')),
        (pl.when(pl.col('Type')=='StartHesitation').then(pl.lit(1)).otherwise(pl.lit(0)).alias('StartHesitation'))
    ]
)

events = events.drop(['Init','Completion','Type'])
events = events.rename({'Init_m': 'Init', 'Completion_m':'Completion'})

events = events.sort(['Id','Init','Completion'], nulls_last=True)

In [8]:
"""## polars SQL can only be used on one dataframe alone, joins are not yet supported
plsql = pl.SQLContext()
#temp_defog = pl.scan_csv(os.path.join(dirname, filename), try_parse_dates=True).filter((pl.col("Valid")) & (pl.col("Task"))).with_columns(pl.lit(filename[0:-4]).alias('Id'))
plsql.register(name='temp_defog', lf=temp_defog)
plsql.register(name='events', lf=events)
temp = plsql.execute(query='select * from temp_defog, events where temp_defog.Id=events.Id limit 10')
temp_df = temp.collect()
temp_df.filter((pl.col('Time')>=np.round(377.635 * 100)) & (pl.col('Time')<=np.round(387.785 * 100)))"""

In [10]:
tasks = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tasks.csv", try_parse_dates=True)

tasks = tasks.with_columns(
    [
        ((pl.col('Begin')*100).round(0).cast(pl.Int64)).alias('Begin_m'), 
        ((pl.col('End')*100).round(0).cast(pl.Int64)).alias('End_m')
    ]
)

tasks = tasks.drop(['Begin','End'])
tasks = tasks.rename({'Begin_m': 'Begin', 'End_m':'End'})

"""print(tasks.shape)
print(tasks.select('Id').n_unique())
tasks.filter(pl.col('Id').is_in(['a8c5a489bd']))"""

"print(tasks.shape)\nprint(tasks.select('Id').n_unique())\ntasks.filter(pl.col('Id').is_in(['a8c5a489bd']))"

In [44]:
defog_train_data = pl.DataFrame()

for dirname, _, filenames in os.walk('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/defog'):
    for filename in filenames:
        
        temp_defog = pl.scan_csv(os.path.join(dirname, filename), try_parse_dates=True).filter((pl.col("Valid")) & (pl.col("Task"))).with_columns(pl.lit(filename[0:-4]).alias('Id'))
        temp_defog = temp_defog.with_columns(
            [
                ((pl.col('AccV')*9.80665).round(5).cast(pl.Float64)).alias('AccV_ms2'), 
                ((pl.col('AccML')*9.80665).round(5).cast(pl.Float64)).alias('AccML_ms2'),
                ((pl.col('AccAP')*9.80665).round(5).cast(pl.Float64)).alias('AccAP_ms2')
            ]
        )
        temp_defog = temp_defog.drop(['AccV','AccML','AccAP'])
        temp_defog = temp_defog.rename({'AccV_ms2': 'AccV', 'AccML_ms2':'AccML', 'AccAP_ms2':'AccAP'})
        
        temp_defog = temp_defog.sort(['Id','Time'], nulls_last=True)
        
        temp_final = dd.sql('SELECT td.Id, td.Time, td.AccV, td.AccML, td.AccAP, td.StartHesitation, td.Turn, td.Walking, e.Kinetic, t.Task \
        FROM temp_defog td \
        left join events e on td.Id = e.Id and td.Time >= e.Init and td.Time <= e.Completion \
        left join tasks t on td.Id = t.Id and td.Time >= t.Begin and td.Time <= t.End').pl()
        
        defog_train_data = pl.concat([defog_train_data, temp_final], how="vertical")
        del temp_final

In [46]:

#defog_train_data.groupby('Id').agg(pl.col('Time').count())
#defog_train_data.select('Task').unique()
"""del defog_train_data
gc.collect()"""
defog_train_data.filter((pl.col('Id')=='a8c5a489bd') & (pl.col('Time')>=82469)).sort('Time')

Id,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Kinetic,Task
str,i64,f64,f64,f64,i64,i64,i64,i64,str
"""a8c5a489bd""",82470,-9.80665,-1.68552,-2.45166,0,0,0,null,"""Hotspot2"""
"""a8c5a489bd""",82471,-9.80665,-1.68552,-2.45166,0,0,0,null,"""Hotspot2"""
"""a8c5a489bd""",82472,-9.80665,-1.61573,-2.54268,0,0,0,null,"""Hotspot2"""
"""a8c5a489bd""",82473,-9.80665,-1.43434,-2.52565,0,0,0,null,"""Hotspot2"""
"""a8c5a489bd""",82474,-9.672,-1.37906,-2.45166,0,0,0,null,"""Hotspot2"""
"""a8c5a489bd""",82475,-9.50019,-1.37906,-2.45166,0,0,0,null,"""Hotspot2"""
"""a8c5a489bd""",82476,-9.50019,-1.37906,-2.45166,0,0,0,null,"""Hotspot2"""
"""a8c5a489bd""",82477,-9.50019,-1.37906,-2.43076,0,0,0,null,"""Hotspot2"""
"""a8c5a489bd""",82478,-9.45107,-1.37906,-2.23024,0,0,0,null,"""Hotspot2"""


In [47]:
defog_train_data.describe()

describe,Id,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Kinetic,Task
str,str,f64,f64,f64,f64,f64,f64,f64,f64,str
"""count""","""4111333""",4.111333e6,4.111333e6,4.111333e6,4.111333e6,4.111333e6,4.111333e6,4.111333e6,4.111333e6,"""4111333"""
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.62572e6,"""0"""
"""mean""",null,91135.571089,-9.168109,-0.012755,-2.216487,0.000021,0.10079,0.017153,0.581294,null
"""std""",null,70209.807728,1.048744,1.372073,2.394782,0.004626,0.301051,0.129841,0.493348,null
"""min""","""02ea782681""",1000.0,-59.08213,-20.74114,-50.19681,0.0,0.0,0.0,0.0,"""4MW"""
"""max""","""f9fc61ce85""",414387.0,43.72162,44.36565,43.03288,1.0,1.0,1.0,1.0,"""Turning-ST"""
"""median""",null,76863.0,-9.2086,0.0,-2.45166,0.0,0.0,0.0,1.0,null


In [ ]:
defog_train_data.select('AccV','AccML','AccAP','Task','StartHesitation','Turn','Walking').corr()